# First Wolfe conditions

## Introduction to optimization and operations research.

Michel Bierlaire


In [ ]:

import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve


In this lab, you will revisit **Newton’s method** on a simple quadratic objective and use it to
practice the **first Wolfe condition** in an inexact line search. You will (i) compute the gradient
and Hessian, (ii) form the **Newton direction** and verify it is a **descent direction** via the
directional derivative, (iii) evaluate the objective along the line x(α) = x₀ + α d, and (iv) check
for which step sizes α the **Wolfe decrease** holds. The goal is to connect the algebra
(∇f, ∇²f, inner products) with practical line-search diagnostics and plots, so you see why Wolfe’s
condition guards sufficient decrease while allowing reasonably large steps.

Consider the unconstrained optimization problem
$$\min_{x \in \mathbb{R}^2} f(x)=4x_1^2-4x_1+x_2^2+2x_2,
$$
and the point $x_0=(0,0)^T$.

- Calculate Newton's direction at $x_0$.
- Verify that it is a descent direction.
- Consider the first Wolfe condition with $\beta_1=0.1$. What are
the values of the step $\alpha$ that verify the condition?

First, implement the function and its derivatives

In [ ]:


def f(x: np.array) -> float:
    """Objective function"""
    # Python starts the numbering at zero
    x_1 = x[0]
    x_2 = x[1]
    result = 4 * x_1 * x_1 - 4 * x_1 + x_2 * x_2 + 2 * x_2
    return result



In [ ]:
x_zero = np.array([0.0, 0.0])
f_zero = f(x_zero)
print(f'f({x_zero}) = {f_zero}')



In [ ]:
def gradient(x: np.array) -> np.array:
    """Gradient of the objective function"""
    x_1 = x[0]
    x_2 = x[1]
    g_1 = 8 * x_1 - 4
    g_2 = 2 * x_2 + 2
    return np.array([g_1, g_2])



In [ ]:
g_zero = gradient(x_zero)
print(f'Gradient of f({x_zero}) = {g_zero}')



In [ ]:
def hessian(x: np.array) -> np.array:
    """Second derivative matrix of the objective function"""
    # In this case, the hessian does not depend on x
    x_1 = x[0]
    x_2 = x[1]
    h_1_1 = 8
    h_1_2 = 0
    h_2_1 = 0
    h_2_2 = 2
    h = np.array([[h_1_1, h_1_2], [h_2_1, h_2_2]])
    return h



In [ ]:
h_zero = hessian(x_zero)
print(f'Hessian of f({x_zero}) =\n{h_zero}')


Note that there exists Python packages for automatic differentiation,
such as ``autograd``or ``jax``.

Calculate Newton's direction

In [ ]:
newton_direction = np.linalg.solve(h_zero, -g_zero)
print(f"Newton's direction: {newton_direction}")


Verify that it is a descent direction.
We calculate the directional derivative.

In [ ]:
directional_derivative = np.inner(newton_direction, g_zero)
print(f'Directional derivative: {directional_derivative}')


It must be negative.

In [ ]:
if directional_derivative < 0:
    print('Descent direction')
else:
    print('Not a descent direction')



Write the function that associates a step alpha along Newton's direction with the value of
the objective function

In [ ]:
def linesearch(alpha: float) -> float:
    """

    :param alpha: step along the direction
    :return: value of the objective function
    """
    new_point = x_zero + alpha * newton_direction
    return f(new_point)



We plot the function.

In [ ]:
alpha_values = np.linspace(0, 2.5, 100)
objective_values = [linesearch(alpha) for alpha in alpha_values]
plt.plot(alpha_values, objective_values)
plt.xlabel('Step alpha')
plt.ylabel('Objective Function Value')
plt.title('Line Search Plot')
plt.grid(True)
plt.show()


Consider the first Wolfe condition with $\beta_1=0.1$.
$$
f(x_\alpha)  \leq f(x_0) + \alpha \beta_1\nabla f(x_0)^Td_N.
$$

In [ ]:
beta_1 = 0.1



Equation of the line defining the first Wolfe condition:
$$
y(\alpha) = f(x_0) + \alpha \beta_1\nabla f(x_0)^Td_N.
$$

In [ ]:
def wolfe(alpha: float) -> float:
    """
    First wolfe condition

    :param alpha: step along the direction
    :return: Wolfe condition
    """
    result = f_zero + alpha * beta_1 * directional_derivative
    return result



Plot the line.

In [ ]:
wolfe_values = [wolfe(alpha) for alpha in alpha_values]
plt.plot(alpha_values, objective_values)
plt.plot(alpha_values, wolfe_values)
plt.xlabel('Step alpha')
plt.ylabel('Objective Function Value')
plt.title('Line Search Plot')
plt.grid(True)
plt.show()


What are the values of the step $\alpha$ that verify the condition?

We need to find the values of alpha such that the difference is positive:
$$
\text{diff}(\alpha) = f(x_0) + \alpha \beta_1\nabla f(x_0)^Td_N - f(x_\alpha) \geq 0.
$$

In [ ]:
def difference(alpha: float) -> float:
    """
    Difference between the first wolfe condition and the function

    :param alpha: step along the direction
    :return: Wolfe condition
    """
    return wolfe(alpha) - linesearch(alpha)



Find the root of that function. Use the function `fsolve` from `scipy`.

In [ ]:
guess = 1.5
root = fsolve(difference, guess)[0]
print(f'Intersection of the function and the Wolfe line: {root:.2g}')


Plot the function with the root.

In [ ]:
plt.plot(alpha_values, objective_values)
plt.plot(alpha_values, wolfe_values)
plt.axvline(root, color='red', linestyle='--')
plt.xlabel('Step alpha')
plt.ylabel('Objective Function Value')
plt.title('Line Search Plot')
plt.show()


The values of the step $\alpha$ that verify the condition are
$$ 0 < \alpha \leq 1.8$$.